In [1]:
import pandas as pd
# Configure some display options for panda frames to ensure we see all columns and rows when needed.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 6)
pd.set_option('display.max_colwidth', -1)

import datetime as dt

# Ensure we embedd the graphs we generate within the Notebook
%matplotlib inline

import numpy as np
from scipy import stats

import seaborn as sns
sns.set(color_codes=True)
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")


In [2]:
time_table = pd.read_csv('/media/sf_data/other_files/PFM_E_TIME_TABLE.csv')

In [3]:
time_table.shape

(887960, 12)

In [4]:
time_table.head()

,SCH_DAY_TYPE_CODE,ROUTE_B_NUMBER,TRN_DISPATCH_TSEC,E_TIMETBL_EFF_DATE,STATION_O_NUMBER,TRAIN_NUMBER,XFER_STATN_NUMBER,DOOR_CLOSE_TIME,DOOR_CLOSE_TSEC,RUN_U_NUMBER,LAST_UPD_DATETIME,DOOR_OPEN_TSEC
0,SA,2,39991,6/22/2003,21,367,1,12:03:45,43425,18,2003-06-20-16:17:35,NaN
1,SA,2,39991,6/22/2003,22,367,1,12:06:20,43580,18,2003-06-20-16:17:35,NaN
2,SA,2,39991,6/22/2003,23,367,1,12:09:41,43781,18,2003-06-20-16:17:35,NaN
3,SA,2,39991,6/22/2003,24,367,1,12:12:40,43960,18,2003-06-20-16:17:35,NaN
4,SA,2,39991,6/22/2003,25,367,1,12:17:01,44221,18,2003-06-20-16:17:35,NaN


In [5]:
time_table.columns

Index([u'SCH_DAY_TYPE_CODE', u'ROUTE_B_NUMBER', u'TRN_DISPATCH_TSEC',
       u'E_TIMETBL_EFF_DATE', u'STATION_O_NUMBER', u'TRAIN_NUMBER',
       u'XFER_STATN_NUMBER', u'DOOR_CLOSE_TIME', u'DOOR_CLOSE_TSEC',
       u'RUN_U_NUMBER', u'LAST_UPD_DATETIME', u'DOOR_OPEN_TSEC'],
      dtype='object')

In [6]:
time_table.SCH_DAY_TYPE_CODE.unique()

array(['SA', 'SU', 'WE'], dtype=object)

IMPORTANT: The E_TIME_TABLE data only has entries for SA, SU and WE.

## Current Time Table
Lets extract only the time-tables we are interested in - for May 2017 onwards.

In [7]:
time_table.E_TIMETBL_EFF_DATE.unique()

array(['6/22/2003', '9/13/2004', '2/9/2004', '9/12/2005', '9/22/2005',
       '1/1/2008', '3/10/2008', '9/14/2009', '9/13/2010', '2/19/2011',
       '9/10/2012', '2/8/2016', '9/14/2015', '3/25/2017'], dtype=object)

In [8]:
selection = (time_table.E_TIMETBL_EFF_DATE == '3/25/2017') & \
(time_table.XFER_STATN_NUMBER == 0) 
current_tt = time_table[selection]

In [9]:
current_tt.shape

(30690, 12)

In [10]:
current_tt.head()

,SCH_DAY_TYPE_CODE,ROUTE_B_NUMBER,TRN_DISPATCH_TSEC,E_TIMETBL_EFF_DATE,STATION_O_NUMBER,TRAIN_NUMBER,XFER_STATN_NUMBER,DOOR_CLOSE_TIME,DOOR_CLOSE_TSEC,RUN_U_NUMBER,LAST_UPD_DATETIME,DOOR_OPEN_TSEC
732220,WE,3,27089,3/25/2017,1,237,0,8:04:57,29097,15,2017-03-27-15:47:42,29072.0
732221,WE,3,27089,3/25/2017,2,237,0,8:00:31,28831,15,2017-03-27-15:47:42,28816.0
732222,WE,3,27089,3/25/2017,3,237,0,7:57:17,28637,15,2017-03-27-15:47:42,28607.0
732223,WE,3,27089,3/25/2017,4,237,0,7:52:54,28374,15,2017-03-27-15:47:42,28359.0
732224,WE,3,27089,3/25/2017,5,237,0,7:49:13,28153,15,2017-03-27-15:47:42,28128.0


In [11]:
tmp = current_tt.groupby([current_tt.SCH_DAY_TYPE_CODE, 
                          current_tt.ROUTE_B_NUMBER, 
                          current_tt.TRAIN_NUMBER]).apply(
                            lambda x: x.sort_values('DOOR_OPEN_TSEC')
                          )

On a given day there can be multiple runs along the same route. The RUN_U_NUMBER and TRN_DISPATCH_TSEC seem to group entries for each run. 

Also it may be possible that the scheduled travel times between stations may differ by trains or runs. So rather than pick one time, we should confirm this by looking at multiple runs.

In [12]:
tmp[['TRN_DISPATCH_TSEC', 'STATION_O_NUMBER', 
     'DOOR_CLOSE_TIME','DOOR_CLOSE_TSEC', 'RUN_U_NUMBER',
     'DOOR_OPEN_TSEC']].head(10)


TRN_DISPATCH_TSEC  \
SCH_DAY_TYPE_CODE ROUTE_B_NUMBER TRAIN_NUMBER                             
SA                1              361          860356  25103               
                                              860355  25103               
                                              860335  25103               
                                              860334  25103               
                                              860333  25103               
                                              860332  25103               
                                              860331  25103               
                                              860330  25103               
                                              860328  25103               
                                              860327  25103               

                                                      STATION_O_NUMBER  \
SCH_DAY_TYPE_CODE ROUTE_B_NUMBER TRAIN_NUMBER                            
SA                1              361          860356  41                 
                                              860355  40                 
                                              860335  19                 
                                              860334  18                 
                                              860333  17                 
                                              860332  16                 
                                              860331  15                 
                                              860330  14                 
                                              860328  12                 
                                              860327  11                 

                                                     DOOR_CLOSE_TIME  \
SCH_DAY_TYPE_CODE ROUTE_B_NUMBER TRAIN_NUMBER                          
SA                1              361          860356  6:58:23          
                                              860355  7:04:30          
                                              860335  7:08:04          
                                              860334  7:13:28          
                                              860333  7:16:05          
                                              860332  7:20:41          
                                              860331  7:25:32          
                                              860330  7:31:05          
                                              860328  7:35:49          
                                              860327  7:39:16          

                                                      DOOR_CLOSE_TSEC  \
SCH_DAY_TYPE_CODE ROUTE_B_NUMBER TRAIN_NUMBER                           
SA                1              361          860356  25103             
                                              860355  25470             
                                              860335  25684             
                                              860334  26008             
                                              860333  26165             
                                              860332  26441             
                                              860331  26732             
                                              860330  27065             
                                              860328  27349             
                                              860327  27556             

                                                      RUN_U_NUMBER  \
SCH_DAY_TYPE_CODE ROUTE_B_NUMBER TRAIN_NUMBER                        
SA                1              361          860356  5              
                                              860355  5              
                                              860335  5              
                                              860334  5              
                                              860333  5              
                                

## Bringing in Station Information

The JSON data names locations with the STATION_CH3_ABBR in the station_list.csv. The E_TIME_TABLE data only references stations with the STATION_O_NUMBER. So in order to be able to match the JSON data with the time-table data, we will need to bring in additional columns of the station_list.csv and join (i.e. pd.merge) them in with the E_TIME_TABLE.

In [13]:
station_list = pd.read_csv('/media/sf_data/other_files/station_list.csv')
station_list.head()

,STATION_O_NUMBER,STATION_P_NUMBER,STATION_Q_NUMBER,STATION_M_NUMBER,STATION_R_NUMBER,STATION_ABBR,STATION_CH3_ABBR,STATION_CH4_ABBR,STATION_CH5_ABBR,STATION_CH8_ABBR,STATION_NAME,PLAT1_DISPL_NUMBER,PLAT2_DISPL_NUMBER,PLAT3_DISPL_NUMBER,PLAT4_DISPL_NUMBER,LINE_NUMBER,LAST_UPD_DATETIME
0,0,NaN,0,0.0,0.0,zr,zro,zero,zero5,zero8,stn zero zone list,1,2,3,4,1,12-Jun-03
1,1,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02
2,2,11.0,11,2.0,20.0,FV,A20,FRVL,FRTVL,FRUITVAL,FRUITVALE,36,37,0,0,5,24-Oct-02
3,3,12.0,12,3.0,21.0,CL,A30,CLSM,COLSM,COLISEUM,COLISEUM,34,35,0,0,5,24-Oct-02
4,4,13.0,13,4.0,22.0,SL,A40,SNLD,SNLAN,SAN LNDR,SAN LEANDRO,32,33,0,0,5,24-Oct-02


In [14]:
station_list.shape

(47, 17)

In [15]:
current_tt.shape

(30690, 12)

In [16]:
current_tt_plus = pd.merge(current_tt, station_list, 
                            left_on='STATION_O_NUMBER', 
                            right_on='STATION_O_NUMBER', 
                            how='left')
current_tt_plus.shape

(30690, 28)

The number of rows match, lets look at some examples.

In [17]:
current_tt_plus['STATION_CH3_ABBR'].head()

0    A10
1    A20
2    A30
3    A40
4    A50
Name: STATION_CH3_ABBR, dtype: object

In [18]:
selection = (current_tt_plus.STATION_CH3_ABBR == 'A10') | \
                (current_tt_plus.STATION_CH3_ABBR == 'A05')
tmp = current_tt_plus[selection]

In [19]:
tmp.head()

,SCH_DAY_TYPE_CODE,ROUTE_B_NUMBER,TRN_DISPATCH_TSEC,E_TIMETBL_EFF_DATE,STATION_O_NUMBER,TRAIN_NUMBER,XFER_STATN_NUMBER,DOOR_CLOSE_TIME,DOOR_CLOSE_TSEC,RUN_U_NUMBER,LAST_UPD_DATETIME_x,DOOR_OPEN_TSEC,STATION_P_NUMBER,STATION_Q_NUMBER,STATION_M_NUMBER,STATION_R_NUMBER,STATION_ABBR,STATION_CH3_ABBR,STATION_CH4_ABBR,STATION_CH5_ABBR,STATION_CH8_ABBR,STATION_NAME,PLAT1_DISPL_NUMBER,PLAT2_DISPL_NUMBER,PLAT3_DISPL_NUMBER,PLAT4_DISPL_NUMBER,LINE_NUMBER,LAST_UPD_DATETIME_y
0,WE,3,27089,3/25/2017,1,237,0,8:04:57,29097,15,2017-03-27-15:47:42,29072.0,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02
18,WE,3,27989,3/25/2017,1,239,0,8:19:57,29997,16,2017-03-27-15:47:42,29972.0,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02
36,WE,3,28889,3/25/2017,1,221,0,8:34:57,30897,17,2017-03-27-15:47:42,30872.0,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02
39,WE,3,31589,3/25/2017,1,227,0,9:19:57,33597,20,2017-03-27-15:47:42,33572.0,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02
57,WE,3,32489,3/25/2017,1,229,0,9:34:57,34497,21,2017-03-27-15:47:42,34472.0,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02


In [20]:
current_tt_plus.dtypes

SCH_DAY_TYPE_CODE      object 
ROUTE_B_NUMBER         int64  
TRN_DISPATCH_TSEC      int64  
E_TIMETBL_EFF_DATE     object 
STATION_O_NUMBER       int64  
TRAIN_NUMBER           int64  
XFER_STATN_NUMBER      int64  
DOOR_CLOSE_TIME        object 
DOOR_CLOSE_TSEC        int64  
RUN_U_NUMBER           int64  
LAST_UPD_DATETIME_x    object 
DOOR_OPEN_TSEC         float64
STATION_P_NUMBER       float64
STATION_Q_NUMBER       int64  
STATION_M_NUMBER       float64
STATION_R_NUMBER       float64
STATION_ABBR           object 
STATION_CH3_ABBR       object 
STATION_CH4_ABBR       object 
STATION_CH5_ABBR       object 
STATION_CH8_ABBR       object 
STATION_NAME           object 
PLAT1_DISPL_NUMBER     int64  
PLAT2_DISPL_NUMBER     int64  
PLAT3_DISPL_NUMBER     int64  
PLAT4_DISPL_NUMBER     int64  
LINE_NUMBER            int64  
LAST_UPD_DATETIME_y    object 
dtype: object

## Adding in Route Information
There is additional meta data in the route information that may be useful. So lets bring that in.

In [21]:
route_info = pd.read_csv('/media/sf_data/other_files/PFM_ROUTE_LIST.csv')
route_info.head()

,ROUTE_B_NUMBER,ROUTE_F_NUMBER,ROUTE_T_NUMBER,ROUTE_ABBR,MAP_COLOR_NAME,HAPROD_NUMBER,COMPASS_DIR_NAME,FROM_STATION,TO_STATION,FROM_STATION_CH3_ABBR,TO_STATION_CH3_ABBR,FROM_STATION_O_NUM,TO_STATION_O_NUM
0,1,2,0,CD,YELLOW,24,WEST,WEST PITTSBURG,S.F. AIRPORT,C80,Y10,41,38
1,2,1,0,DC,YELLOW,42,EAST,S.F. AIRPORT,WEST PITTSBURG,Y10,C80,38,41
2,3,4,0,FR,ORANGE,13,NORTH,FREMONT,RICHMOND,A90,R60,9,25
3,4,3,0,RF,ORANGE,31,SOUTH,RICHMOND,FREMONT,R60,A90,25,9
4,5,6,0,FD,GREEN,14,WEST,FREMONT,DALY CITY,A90,M90,9,34


In [22]:
current_tt_plus = pd.merge(current_tt_plus, route_info, 
                            left_on='ROUTE_B_NUMBER', 
                            right_on='ROUTE_B_NUMBER', 
                            how='left')
current_tt_plus.shape

(30690, 40)

In [23]:
current_tt_plus.head()

,SCH_DAY_TYPE_CODE,ROUTE_B_NUMBER,TRN_DISPATCH_TSEC,E_TIMETBL_EFF_DATE,STATION_O_NUMBER,TRAIN_NUMBER,XFER_STATN_NUMBER,DOOR_CLOSE_TIME,DOOR_CLOSE_TSEC,RUN_U_NUMBER,LAST_UPD_DATETIME_x,DOOR_OPEN_TSEC,STATION_P_NUMBER,STATION_Q_NUMBER,STATION_M_NUMBER,STATION_R_NUMBER,STATION_ABBR,STATION_CH3_ABBR,STATION_CH4_ABBR,STATION_CH5_ABBR,STATION_CH8_ABBR,STATION_NAME,PLAT1_DISPL_NUMBER,PLAT2_DISPL_NUMBER,PLAT3_DISPL_NUMBER,PLAT4_DISPL_NUMBER,LINE_NUMBER,LAST_UPD_DATETIME_y,ROUTE_F_NUMBER,ROUTE_T_NUMBER,ROUTE_ABBR,MAP_COLOR_NAME,HAPROD_NUMBER,COMPASS_DIR_NAME,FROM_STATION,TO_STATION,FROM_STATION_CH3_ABBR,TO_STATION_CH3_ABBR,FROM_STATION_O_NUM,TO_STATION_O_NUM
0,WE,3,27089,3/25/2017,1,237,0,8:04:57,29097,15,2017-03-27-15:47:42,29072.0,10.0,10,1.0,19.0,LM,A10,LKMT,LKMER,LAKE MRT,LAKE MERRITT,38,39,0,0,5,24-Oct-02,4,0,FR,ORANGE,13,NORTH,FREMONT,RICHMOND,A90,R60,9,25
1,WE,3,27089,3/25/2017,2,237,0,8:00:31,28831,15,2017-03-27-15:47:42,28816.0,11.0,11,2.0,20.0,FV,A20,FRVL,FRTVL,FRUITVAL,FRUITVALE,36,37,0,0,5,24-Oct-02,4,0,FR,ORANGE,13,NORTH,FREMONT,RICHMOND,A90,R60,9,25
2,WE,3,27089,3/25/2017,3,237,0,7:57:17,28637,15,2017-03-27-15:47:42,28607.0,12.0,12,3.0,21.0,CL,A30,CLSM,COLSM,COLISEUM,COLISEUM,34,35,0,0,5,24-Oct-02,4,0,FR,ORANGE,13,NORTH,FREMONT,RICHMOND,A90,R60,9,25
3,WE,3,27089,3/25/2017,4,237,0,7:52:54,28374,15,2017-03-27-15:47:42,28359.0,13.0,13,4.0,22.0,SL,A40,SNLD,SNLAN,SAN LNDR,SAN LEANDRO,32,33,0,0,5,24-Oct-02,4,0,FR,ORANGE,13,NORTH,FREMONT,RICHMOND,A90,R60,9,25
4,WE,3,27089,3/25/2017,5,237,0,7:49:13,28153,15,2017-03-27-15:47:42,28128.0,14.0,14,5.0,23.0,BF,A50,BYFR,BAYFR,BAY FAIR,BAY FAIR,30,31,0,0,5,24-Oct-02,4,0,FR,ORANGE,13,NORTH,FREMONT,RICHMOND,A90,R60,9,25


## Constructing the Schedule

Now, lets construct the actual schedule. We'll group, sort and select a subset of columns of interest. Once we have that, we will then iterate through the rows building up a map of expected travel times between station (i.e. STATION_CH3_ABBR).

On a given day there can be multiple runs along the same route. The RUN_U_NUMBER and TRN_DISPATCH_TSEC seem to group entries for each run.

In [24]:
tt_of_interest = current_tt_plus.groupby([current_tt_plus.SCH_DAY_TYPE_CODE, 
                          current_tt_plus.ROUTE_B_NUMBER, 
                          current_tt_plus.TRAIN_NUMBER,
                          current_tt_plus.RUN_U_NUMBER]).apply(
                            lambda x: x.sort_values('DOOR_OPEN_TSEC')
                          )

In [25]:
tt_of_interest = tt_of_interest[['SCH_DAY_TYPE_CODE','ROUTE_B_NUMBER','TRAIN_NUMBER','RUN_U_NUMBER',
 'STATION_O_NUMBER','STATION_CH3_ABBR','STATION_NAME',
 'DOOR_CLOSE_TIME','DOOR_CLOSE_TSEC','DOOR_OPEN_TSEC',
 'LINE_NUMBER','ROUTE_ABBR','MAP_COLOR_NAME',
 'FROM_STATION_O_NUM','TO_STATION_O_NUM',
 'FROM_STATION_CH3_ABBR','TO_STATION_CH3_ABBR',
 'FROM_STATION','TO_STATION']]

Let us also convert the DOOR_CLOSE_TIME into a datetime object. Note that the date will be the date of when the code is run (i.e. now). But since we are only really interested in the time and duration, we will ignore the date elements.

In [26]:
tt_of_interest['DOOR_CLOSE_TIME'] = pd.to_datetime(tt_of_interest['DOOR_CLOSE_TIME'],
                                              infer_datetime_format=True, errors='ignore')
print tt_of_interest.dtypes

SCH_DAY_TYPE_CODE        object        
ROUTE_B_NUMBER           int64         
TRAIN_NUMBER             int64         
RUN_U_NUMBER             int64         
STATION_O_NUMBER         int64         
STATION_CH3_ABBR         object        
STATION_NAME             object        
DOOR_CLOSE_TIME          datetime64[ns]
DOOR_CLOSE_TSEC          int64         
DOOR_OPEN_TSEC           float64       
LINE_NUMBER              int64         
ROUTE_ABBR               object        
MAP_COLOR_NAME           object        
FROM_STATION_O_NUM       int64         
TO_STATION_O_NUM         int64         
FROM_STATION_CH3_ABBR    object        
TO_STATION_CH3_ABBR      object        
FROM_STATION             object        
TO_STATION               object        
dtype: object


In [27]:
tt_of_interest['SCH_DAY_TYPE_CODE'].unique()

array(['SA', 'SU', 'WE'], dtype=object)

In [28]:
print tt_of_interest.index.get_level_values(0).unique()
print tt_of_interest.index.get_level_values(1).unique()
print tt_of_interest.index.get_level_values(2).unique()
print tt_of_interest.index.get_level_values(3).unique()
print tt_of_interest.index.get_level_values(4).unique()
print tt_of_interest.index.names

['SA' 'SU' 'WE']
[ 1  2  3  4  5  6  7  8 11 12 13 14]
[361 363 365 367 369 371 373 375 377 379 221 223 225 227 229 231 233 235
 101 103 105 107 109 111 113 441 443 445 447 449 451 453 501 503 505 507
 509 511 513 901 321 323 325 327 329 331 333 335 381 383 385 237 239 115
 117 119 455 457 459 461 515 517 519]
[ 5 15 25 35 45 55  6 16 26 36 46 56  7 17 27 37 47  8 18 28 38 48  9 19 29
 39 49 10 20 30 40 50  1 11 21 31 41 51  2 12 22 32 42 52  3 13 23 33 43 53
  4 14 24 34 44 54 65 67 69 71 73 75 70 85 95 72 86 96 74 57 76 87 58 77 59
 78 88 60 79 89 61 80 90 62 81 91 63 82 92 64 83 93 66 68 84 94]
[17786 17785 17771 ..., 16440 16459 16460]
[u'SCH_DAY_TYPE_CODE', u'ROUTE_B_NUMBER', u'TRAIN_NUMBER', u'RUN_U_NUMBER', None]


In [29]:
tt_of_interest.to_csv('tt_of_interest_no_index.csv', index=False)

In [30]:
tt_of_interest.to_csv('tt_of_interest_with_index.csv')

## Construct Expected Travel Times Map
Given the schedule, we can not construct the expected travel times between stations. Also it may be possible that the scheduled travel times between stations may differ by trains or runs. So rather than pick one time, we should confirm this by looking at multiple runs.

In [31]:
TIME_DELTA_24h = dt.timedelta(hours=24)

def is_suitable_for_schedule_record(row_a, row_b):
    """Returns True if the following conditions are met:
      * SCH_DAY_TYPE_CODE match
      * ROUTE_B_NUMBER match
      * TRAIN_NUMBER match
      * RUN_U_NUMBER match
      * STATION_O_NUMBER are different
      * DOOR_CLOSE_TIME are within 24h and maintain order.
      """
    if row_a['SCH_DAY_TYPE_CODE'] != row_b['SCH_DAY_TYPE_CODE']:
        # Entries from different day
        return False
    if row_a['ROUTE_B_NUMBER'] != row_b['ROUTE_B_NUMBER']:
        # Entries from different route
        return False
    if row_a['TRAIN_NUMBER'] != row_b['TRAIN_NUMBER']:
        # Entries from different trains
        return False
    if row_a['RUN_U_NUMBER'] != row_b['RUN_U_NUMBER']:
        # Entries from different run
        return False
    if row_a['STATION_O_NUMBER'] == row_b['STATION_O_NUMBER']:
        # Entries for the same station
        return False
    
    time_diff = row_b['DOOR_CLOSE_TIME'] - row_a['DOOR_CLOSE_TIME']
    if time_diff > TIME_DELTA_24h:
        # This check ensure we are only considering entries within a 24hour period.
        return False
    if time_diff.total_seconds() < 0:
        # Ignore the wrap-around over the midnight giving us negative values
        return False
    
    return True

class ScheduleRecord(object):
    def __init__(self, row_a, row_b):
        """Represents a ScheduleRecord between two locations, including the start, end times as well as durations."""
        self._location_a = row_a['STATION_CH3_ABBR']
        self._location_b = row_b['STATION_CH3_ABBR']
        self._start_time = row_a['DOOR_CLOSE_TIME']
        self._end_time = row_b['DOOR_CLOSE_TIME']
        self._travel_time_secs = (self._end_time - self._start_time).total_seconds()
        
        self._day_of_week = row_a['SCH_DAY_TYPE_CODE']
        self._route_b_number  = row_a['ROUTE_B_NUMBER']
        self._run_number = row_a['RUN_U_NUMBER']
        
    def __str__(self):
        ret = "{} @ {} --[{}]--> {} @ {} (Day={}, Route_B={}, Run={})".format(self._location_a, 
                                                                              self._start_time, 
                                                                              self._travel_time_secs,
                                                                              self._location_b, 
                                                                              self._end_time,
                                                                              self._day_of_week,
                                                                              self._route_b_number,
                                                                              self._run_number)
        return ret
    
    def __repr__(self):
        return self.__str__()

        
def add_map_entry_a2b_list(map_to_update, key_a, key_b, list_entry):
    """Adds list_entry safely in the map for (key_a, key_b).
       If the list_entry is a list itself, it concatenates the list.
       Otherwise if it is an object, it appends to the existing list.
       """
    map_inner = dict()
    
    if key_a in map_to_update:
        # If a nested dict exists, we replace map_inner with it.
        map_inner = map_to_update[key_a]
    else:
        # If not, make sure we inject map_inner, or subsequent check for list will fail.
        map_to_update[key_a] = map_inner

    list_objects = list()
    if key_b in map_inner:
        # If a list exist, we replace list with it
        list_objects = map_inner[key_b]

    # Update the list
    if isinstance(list_entry, list):
        list_objects = list_objects + list_entry
    else:
        list_objects.append(list_entry)
        
    # Put it back in
    map_to_update[key_a][key_b] = list_objects

In [32]:
map_a2b_to_list_times = dict()

row_a = None
row_b = None

num_rows = 0
num_groups = 0

for (day_code, route_b, train_no, run_u_number), df_group in tt_of_interest.groupby(level=[0,1,2,3]):
    print ".",
    for index, row in df_group.iterrows():
        num_rows += 1
        if row_a is None:
            row_a = row
            # Skipping the rest of the processing
            continue
        elif row_b is None:
            row_b = row

            # Now do some processing on the record pair, IF the conditions are met
            if is_suitable_for_schedule_record(row_a, row_b):
                schedule_record = ScheduleRecord(row_a, row_b)
                
                add_map_entry_a2b_list(map_a2b_to_list_times, schedule_record._location_a, 
                                       schedule_record._location_b, schedule_record)
            # Otherwise we simply skip this pair and advance the window
            # final step to advance
            row_a = row_b
            row_b = None
    num_groups += 1

print "Processed {} groups and {} rows".format(num_groups, num_rows)


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [33]:
import pprint

pprint.pprint(map_a2b_to_list_times['M90']['M80'])

[M90 @ 2017-08-15 05:43:10 --[196.0]--> M80 @ 2017-08-15 05:46:26 (Day=SA, Route_B=2, Run=1),
 M90 @ 2017-08-15 09:03:29 --[211.0]--> M80 @ 2017-08-15 09:07:00 (Day=SA, Route_B=2, Run=11),
 M90 @ 2017-08-15 12:23:29 --[211.0]--> M80 @ 2017-08-15 12:27:00 (Day=SA, Route_B=2, Run=21),
 M90 @ 2017-08-15 15:43:29 --[211.0]--> M80 @ 2017-08-15 15:47:00 (Day=SA, Route_B=2, Run=31),
 M90 @ 2017-08-15 19:03:29 --[211.0]--> M80 @ 2017-08-15 19:07:00 (Day=SA, Route_B=2, Run=41),
 M90 @ 2017-08-15 22:23:29 --[211.0]--> M80 @ 2017-08-15 22:27:00 (Day=SA, Route_B=2, Run=51),
 M90 @ 2017-08-15 06:03:29 --[211.0]--> M80 @ 2017-08-15 06:07:00 (Day=SA, Route_B=2, Run=2),
 M90 @ 2017-08-15 09:23:29 --[211.0]--> M80 @ 2017-08-15 09:27:00 (Day=SA, Route_B=2, Run=12),
 M90 @ 2017-08-15 12:43:29 --[211.0]--> M80 @ 2017-08-15 12:47:00 (Day=SA, Route_B=2, Run=22),
 M90 @ 2017-08-15 16:03:29 --[211.0]--> M80 @ 2017-08-15 16:07:00 (Day=SA, Route_B=2, Run=32),
 M90 @ 2017-08-15 19:23:29 --[211.0]--> M80 @ 2017-0

### Summary Statistics
Now we generate summary statistics for each leg. There are some examples where runs have different schedule times. For example, Run 1 on Route 12 is faster than any other entry.

* M90 @ 2017-08-15 21:50:41 --[211.0]--> M80 @ 2017-08-15 21:54:12 (Day=SA, Route_B=12, Run=49),
* M90 @ 2017-08-15 05:50:41 --[196.0]--> M80 @ 2017-08-15 05:53:57 (Day=SA, Route_B=12, Run=1),




In [34]:
class TravelStatistics(object):
    def __init__(self, start, end, list_travel_times):
        """Given a TravelRecord with multiple trips, generates statistics including:
         * stats.describe.
         * median (i.e. p50
         * p90
         * p99
         * p99.9
         """
        self._start = start
        self._end = end
        np_array = np.sort(np.fromiter(list_travel_times, np.float))
        
        percentiles = np.percentile(np_array, [50, 75, 90, 95, 99])
        self._p50 = percentiles[0]
        self._p75 = percentiles[1]
        self._p90 = percentiles[2]
        self._p95 = percentiles[3]
        self._p99 = percentiles[4]
        
        stats_obj = stats.describe(np_array)
        self._nobs = stats_obj.nobs
        self._min = stats_obj.minmax[0]
        self._max = stats_obj.minmax[1]
        self._mean = stats_obj.mean
        self._variance = stats_obj.variance
        self._skewness = stats_obj.skewness
        self._kurtosis = stats_obj.kurtosis
        
        self._mode = stats.mode(np_array)

        
    def __str__(self):
        ret = "start={}, end={}".format(self._start, self._end)
        ret += ", nobs={}, min={}, max={}, mode={}".format(self._nobs, self._min, self._max, self._mode)
        ret += ", p50={}, p75={}, p90={}, p95={}, p99={}".format(self._p50, self._p75, self._p90, self._p95, self._p99)
        ret += ", mean={}, variance={}".format(self._mean, self._variance)
        ret += ", skewness={}, kurtosis={}".format(self._skewness, self._kurtosis)        
        return ret

    def __repr__(self):
        return self.__str__() 

def get_travel_statistics(map_a2b_to_list_times):
    """Returns a map with the TravelStatistics objects for each (key1,key2) pairs in the given map"""
    map_stats_a2b_list = dict()
    num_records = 0
    for start in map_a2b_to_list_times.keys():
        for end in map_a2b_to_list_times[start].keys():
            list_travel_recs = map_a2b_to_list_times[start][end]
            list_times = [x._travel_time_secs for x in list_travel_recs]
            travel_stats = TravelStatistics(start, end, list_times)
            add_map_entry_a2b_list(map_stats_a2b_list, start, end, travel_stats)
            num_records += 1
    print "Generated TravelStatistics for {} records.".format(num_records)
    return map_stats_a2b_list

In [35]:
map_a2b_stats = get_travel_statistics(map_a2b_to_list_times)

Generated TravelStatistics for 92 records.


In [36]:
pprint.pprint(map_a2b_stats['M90']['M80'])

[start=M90, end=M80, nobs=540, min=196.0, max=211.0, mode=ModeResult(mode=array([ 211.]), count=array([537])), p50=211.0, p75=211.0, p90=211.0, p95=211.0, p99=211.0, mean=210.916666667, variance=1.24536178108, skewness=-13.304344651, kurtosis=175.005586592]


## Export Map for Analysis

In [37]:
import pickle
with open('schedule_travel_time_stats.pickle', 'wb') as handle:
    pickle.dump(map_a2b_stats, handle)